# Wandb : Tensorflow

* 0714fddca3d8c2ab50ab0e32cf0de1bdebc97316

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers, optimizers, datasets
import numpy as np

In [2]:
(images_train,labels_train), (images_test,labels_test) = datasets.mnist.load_data()
print('----------------train----------------')
print('images:',images_train.shape, images_train.min(), images_train.max())
print('labels:',labels_train.shape)
print('----------------test----------------')
print('images:',images_test.shape, images_test.min(), images_test.max())
print('labels:',labels_test.shape)

----------------train----------------
images: (60000, 28, 28) 0 255
labels: (60000,)
----------------test----------------
images: (10000, 28, 28) 0 255
labels: (10000,)


In [3]:
# [0,255] to [0,1]

images_train = images_train/255.
images_test = images_test/255.

In [4]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jack04215. Use `wandb login --relogin` to force relogin


True

In [5]:
args = dict(
    total_epochs=3,
    batch_size=128,
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics='accuracy',
    learning_rate=0.005,
    num_classes=10,
    hidden_size=[128,256])

In [6]:
wandb.init(project='fast_campus_deep_learning(Tensorflow)')
wandb.config.update(args)

In [7]:
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(args['hidden_size'][0], activation='relu'),
    layers.Dense(args['hidden_size'][1], activation='relu'),
    layers.Dense(args['num_classes'], activation='softmax')
])

In [8]:
model.compile(optimizer=args['optimizer'],
              loss=args['loss'],
              metrics=args['metrics'])

* callbacks=[WandbCallback(data_type="image",labels=labels_train)]

In [9]:
model.fit(images_train, labels_train, validation_data=(images_test, labels_test), epochs=args['total_epochs'], batch_size=args['batch_size'], callbacks=[WandbCallback(data_type="image",labels=labels_train)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/3
469/469 [==============================] - 20s 38ms/step - loss: 0.3054 - accuracy: 0.9132 - val_loss: 0.1433 - val_accuracy: 0.9562 - _timestamp: 1651907382.0000 - _runtime: 74.0000
Epoch 2/3
469/469 [==============================] - 14s 29ms/step - loss: 0.1189 - accuracy: 0.9651 - val_loss: 0.1050 - val_accuracy: 0.9653 - _timestamp: 1651907395.0000 - _runtime: 87.0000
Epoch 3/3
469/469 [==============================] - 12s 26ms/step - loss: 0.0815 - accuracy: 0.9741 - val_loss: 0.0886 - val_accuracy: 0.9725 - _timestamp: 1651907407.0000 - _runtime: 99.0000


In [10]:
loss, accuracy = model.evaluate(images_test, labels_test)
wandb.log({'test_loss':loss})
wandb.log({'test_accuracy':accuracy})

313/313 [==============================] - 2s 6ms/step - loss: 0.0886 - accuracy: 0.9725
